In [34]:
%load_ext autoreload
%autoreload 2
from finetune import *
from tqdm import tqdm
from huggingface_hub import get_full_repo_name
from huggingface_hub import Repository
from tqdm import tqdm
import torch
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
toy_args

{'cultural_corpus': 'yo-bm25-10000',
 'pretrained_model': 'FacebookAI/xlm-roberta-base',
 'corpus_chunk_size': 128,
 'wwm_probability': 0.15,
 'batch_size': 64}

In [36]:
tokenizer = mk_tokenizer(toy_args)
mlm_model, mc_model = mk_models(toy_args)

mask token id is 250001 and mask token is <mask>
tokenizer max len 512
model config: XLMRobertaConfig {
  "_name_or_path": "FacebookAI/xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mlm_model num param: 278295186
mc_model num param: 278044417


In [37]:
fill_mask(toy_args, mlm_model, tokenizer, f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}.")

input: The cat said <mask>.
pred 0: The cat said _:_.
pred 1: The cat said _,_.
pred 2: The cat said _that_.
pred 3: The cat said _:_.
pred 4: The cat said _._.
pred 5: The cat said _it_.
pred 6: The cat said _he_.
pred 7: The cat said _"_.
pred 8: The cat said _to_.
pred 9: The cat said _..._.


In [38]:
toy_text = f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}."
toy_input = tokenizer(toy_text, return_tensors="pt")
mlm_model(**toy_input).keys()

odict_keys(['logits', 'hidden_states'])

In [39]:
lm_corpus, corpus_mask_collator, corpus_wwm_collator = mk_corpus(toy_args, tokenizer)
lm_corpus

Map: 100%|██████████| 42/42 [00:00<00:00, 13685.58 examples/s]

index 0 train example:
{'input_ids': [0, 18826, 7, 5154, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 23
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2
    })
})

In [40]:
figqa_datasets, figqa_data_collator = mk_figqa_dataset(toy_args, tokenizer)
figqa_datasets

Map: 100%|██████████| 2/2 [00:00<00:00, 763.16 examples/s]

figqa data looks like:
train: {'labels': 0, 'input_ids': [[0, 581, 7515, 2804, 163, 8770, 2, 2, 163, 8770, 2], [0, 581, 7515, 2804, 163, 8770, 2, 2, 4323, 4390, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> The cat said meow</s></s> meow</s>', '<s> The cat said meow</s></s> woof</s>']
val: {'labels': 0, 'input_ids': [[0, 581, 7515, 2804, 163, 8770, 2, 2, 163, 8770, 2], [0, 581, 7515, 2804, 163, 8770, 2, 2, 4323, 4390, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> The cat said meow</s></s> meow</s>', '<s> The cat said meow</s></s> woof</s>']
test: {'labels': 0, 'input_ids': [[0, 62, 45730, 7515, 7, 1884, 47, 3249, 83, 163, 8770, 2, 2, 163, 8770, 2], [0, 62, 45730, 7515, 7, 1884, 47, 3249, 83, 163, 8770, 2, 2, 4323, 4390, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
decoded: ['<s> A 

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 80
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
})

## MLM train

In [37]:
training_args = TrainingArguments(
    output_dir=f"xlmr-finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    per_device_train_batch_size=toy_args['batch_size'],
    per_device_eval_batch_size=toy_args['batch_size'],
    logging_steps=1,
    num_train_epochs=10,
    use_cpu=True,
    use_mps_device=False,
)
trainer = Trainer(
    model=mlm_model,
    args=training_args,
    train_dataset=lm_corpus["train"],
    eval_dataset=lm_corpus["val"],
    data_collator=corpus_mask_collator,
    tokenizer=tokenizer,
)

/Users/chaosarium/anaconda3/envs/multi/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [38]:
trainer.evaluate()

100%|██████████| 1/1 [00:00<00:00, 405.83it/s]


{'eval_loss': 26.20921516418457,
 'eval_runtime': 0.3717,
 'eval_samples_per_second': 2.69,
 'eval_steps_per_second': 2.69}

In [39]:
trainer.train()

 10%|█         | 1/10 [00:11<01:47, 11.99s/it]

{'loss': 24.9817, 'grad_norm': 804.8196411132812, 'learning_rate': 0.00045000000000000004, 'epoch': 1.0}


                                              
 10%|█         | 1/10 [00:12<01:47, 11.99s/it]

{'eval_loss': 11.300202369689941, 'eval_runtime': 0.293, 'eval_samples_per_second': 3.413, 'eval_steps_per_second': 3.413, 'epoch': 1.0}


 20%|██        | 2/10 [00:23<01:34, 11.86s/it]

...

100%|██████████| 10/10 [01:54<00:00, 11.27s/it]

{'loss': 2.9403, 'grad_norm': 22.08307647705078, 'learning_rate': 0.0, 'epoch': 10.0}


                                               
100%|██████████| 10/10 [01:54<00:00, 11.49s/it]

{'eval_loss': 2.725783586502075, 'eval_runtime': 0.283, 'eval_samples_per_second': 3.534, 'eval_steps_per_second': 3.534, 'epoch': 10.0}
{'train_runtime': 114.9341, 'train_samples_per_second': 1.392, 'train_steps_per_second': 0.087, 'train_loss': 6.580933213233948, 'epoch': 10.0}


TrainOutput(global_step=10, training_loss=6.580933213233948, metrics={'train_runtime': 114.9341, 'train_samples_per_second': 1.392, 'train_steps_per_second': 0.087, 'train_loss': 6.580933213233948, 'epoch': 10.0})

In [40]:
fill_mask(toy_args, mlm_model, tokenizer, f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}.")

input: The cat said <mask>.
pred 0: The cat said _me_.
pred 1: The cat said _ow_.
pred 2: The cat said _ken_.
pred 3: The cat said _s_.
pred 4: The cat said _Cat_.
pred 5: The cat said _say_.
pred 6: The cat said _said_.
pred 7: The cat said _are_.
pred 8: The cat said _To_.
pred 9: The cat said _The_.


## MLM but with train loop

In [41]:
no_word_ids_lm_corpus = lm_corpus.remove_columns(["word_ids"])

In [42]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = corpus_mask_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}
eval_dataset = no_word_ids_lm_corpus["val"].map(
    insert_random_mask,
    batched=True,
    remove_columns=no_word_ids_lm_corpus["val"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)
print(eval_dataset[0])
eval_dataset

Map: 100%|██████████| 2/2 [00:00<00:00, 42.91 examples/s]

{'input_ids': [0, 581, 7515, 2804, 250001, 8770, 2, 0, 250001, 7515, 187318, 163, 8770, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 18826, 7, 5154, 163, 250001, 2, 0, 18826, 7, 5154, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0, 18826, 7, 250001, 250001, 8770, 2, 0, 250001, 7515, 2804, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 38748, 8770, 2, 0, 18826, 250001, 5154, 250001, 8770, 2, 0, 717, 1098, 250001, 7, 621, 221, 163, 250001, 2, 0, 581, 7515, 250001, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 221, 163, 8770, 2, 0, 717, 1098, 52825, 250001, 621, 221, 163, 8770, 2, 0, 717, 1098, 52825, 7, 621, 250001, 163, 8770, 2, 0, 581, 7515, 2804, 163, 8770, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})

In [43]:
train_dataloader = DataLoader(
    no_word_ids_lm_corpus["train"],
    shuffle=True,
    batch_size=toy_args['batch_size'],
    collate_fn=corpus_mask_collator,
)
eval_dataloader = DataLoader(eval_dataset, batch_size=toy_args['batch_size'], collate_fn=default_data_collator)

In [44]:
optimizer = torch.optim.AdamW(mlm_model.parameters(), lr=5e-4)
lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=3,
    num_training_steps=30,
)
# model_name = "testtest"
# repo_name = get_full_repo_name(model_name)
# repo_name
# output_dir = model_name
# repo = Repository(output_dir, clone_from=repo_name)

In [47]:
accelerator1 = Accelerator(cpu=False)
mlm_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator1.prepare(mlm_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler)
mlm_model.device, train_dataloader.device, eval_dataloader.device

(device(type='cpu'), device(type='cpu'), device(type='cpu'))

In [48]:
for epoch in tqdm(range(10)):
    print('training...')
    mlm_model.train()
    for batch in tqdm(train_dataloader):
        outputs = mlm_model(**batch)
        loss = outputs.loss
        accelerator1.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar.update(1)

    print('evaluating...')
    mlm_model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = mlm_model(**batch)

        loss = outputs.loss
        losses.append(accelerator1.gather(loss.repeat(toy_args['batch_size'])))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try: perplexity = math.exp(torch.mean(losses))
    except OverflowError: perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")
    fill_mask(toy_args, mlm_model, tokenizer, f"The cat said {tokenizer.decode(tokenizer.mask_token_id)}.")
    # Save and upload
    # accelerator.wait_for_everyone()
    # unwrapped_model = accelerator.unwrap_model(model)
    # unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    # if accelerator.is_main_process:
    #     tokenizer.save_pretrained(output_dir)
    #     repo.push_to_hub(
    #         commit_message=f"Training in progress epoch {epoch}", blocking=False
    #     )

  0%|          | 0/10 [00:00<?, ?it/s]

training...


100%|██████████| 1/1 [00:21<00:00, 21.12s/it]


evaluating...


 10%|█         | 1/10 [00:21<03:16, 21.82s/it]

...

>>> Epoch 8: Perplexity: 19.470916677189024
input: The cat said <mask>.
pred 0: The cat said _izer_.
pred 1: The cat said _me_.
pred 2: The cat said _s_.
pred 3: The cat said _so_.
pred 4: The cat said _ken_.
pred 5: The cat said _cat_.
pred 6: The cat said _ow_.
pred 7: The cat said _said_.
pred 8: The cat said _To_.
pred 9: The cat said _are_.
training...


100%|██████████| 1/1 [00:23<00:00, 23.83s/it]


evaluating...


100%|██████████| 10/10 [04:00<00:00, 24.02s/it]

>>> Epoch 9: Perplexity: 18.556267452002295
input: The cat said <mask>.
pred 0: The cat said _izer_.
pred 1: The cat said _me_.
pred 2: The cat said _so_.
pred 3: The cat said _ow_.
pred 4: The cat said _s_.
pred 5: The cat said _ken_.
pred 6: The cat said _cat_.
pred 7: The cat said _Cat_.
pred 8: The cat said _To_.
pred 9: The cat said _say_.


## FigQA training

In [47]:
train_dataset = figqa_datasets["train"]
eval_dataset = figqa_datasets["val"]

train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=figqa_data_collator, batch_size=toy_args['batch_size'])
eval_dataloader = DataLoader(eval_dataset, collate_fn=figqa_data_collator, batch_size=toy_args['batch_size'])

In [48]:
optimizer = torch.optim.AdamW(mc_model.parameters(), lr=2e-5)
lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=3,
    num_training_steps=30,
)

In [49]:
mc_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(mc_model, optimizer, train_dataloader, eval_dataloader, lr_scheduler)

In [50]:
metric = evaluate.load("accuracy")

In [63]:
for batch in train_dataloader:
    break
batch['input_ids'][0], tokenizer.decode(batch['input_ids'][0][0])

(tensor([[   0,  581, 7515, 2804,  163, 8770,    2,    2,  163, 8770,    2],
         [   0,  581, 7515, 2804,  163, 8770,    2,    2, 4323, 4390,    2]]),
 '<s> The cat said meow</s></s> meow</s>')

In [51]:
completed_steps = 0

for epoch in range(0, 10):
    mc_model.train()
    mc_model, total_loss, completed_steps = train_model(train_dataloader, mc_model, accelerator, optimizer, lr_scheduler, toy_args, completed_steps, checkpointing_steps=500)
    eval_metric = eval_model(mc_model, eval_dataloader, metric, accelerator, epoch, toy_args)

    print({
        "accuracy": eval_metric,
        "train_loss": total_loss.item() / len(train_dataloader),
        "epoch": epoch,
        "step": completed_steps,
    })

    if False:
        if toy_args["push_to_hub"] and epoch < toy_args["num_train_epochs"] - 1:
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(mc_model)
            unwrapped_model.save_pretrained(
                toy_args["output_dir"], is_main_process=accelerator.is_main_process, save_function=accelerator.save
            )
            if accelerator.is_main_process:
                tokenizer.save_pretrained(toy_args["output_dir"])
                # repo.push_to_hub(
                #     commit_message=f"Training in progress epoch {epoch}", blocking=False, auto_lfs_prune=True
                # )

        if toy_args["checkpointing_steps"] == "epoch":
            output_dir = f"epoch_{epoch}"
            if toy_args["output_dir"] is not None:
                output_dir = os.path.join(toy_args["output_dir"], output_dir)
            accelerator.save_state(output_dir)

if False:
    if toy_args["output_dir"] is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(mc_model)
        unwrapped_model.save_pretrained(
            toy_args["output_dir"], is_main_process=accelerator.is_main_process, save_function=accelerator.save
        )
        if accelerator.is_main_process:
            tokenizer.save_pretrained(toy_args["output_dir"])
            # if args["push_to_hub"]:
            #     repo.push_to_hub(commit_message="End of training", auto_lfs_prune=True)
        with open(os.path.join(toy_args["output_dir"], "all_results.json"), "w") as f:
            json.dump({"eval_accuracy": eval_metric["accuracy"]}, f)

eval_metric["accuracy"]


 50%|█████     | 1/2 [00:04<00:04,  4.11s/it]

tensor(0.6984, grad_fn=<NllLossBackward0>)


100%|██████████| 2/2 [00:05<00:00,  2.74s/it]


tensor(0.7041, grad_fn=<NllLossBackward0>)
epoch 0: {'accuracy': 0.75}
{'accuracy': {'accuracy': 0.75}, 'train_loss': 0.3520698547363281, 'epoch': 0, 'step': 2}


 50%|█████     | 1/2 [00:03<00:03,  3.59s/it]

...

 50%|█████     | 1/2 [00:03<00:03,  3.81s/it]

tensor(0.5493, grad_fn=<NllLossBackward0>)


100%|██████████| 2/2 [00:05<00:00,  2.75s/it]

tensor(0.5318, grad_fn=<NllLossBackward0>)
epoch 9: {'accuracy': 1.0}
{'accuracy': {'accuracy': 1.0}, 'train_loss': 0.265889436006546, 'epoch': 9, 'step': 20}


1.0